In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
torch.manual_seed(1)

In [10]:
data = pd.read_csv("/home/dataset/FINAL_SEQUENCES_MATRIX.tsv", sep='\t')

In [11]:
a = {}

set_of_hash = set()

for index, row in data.iterrows():
    temp = [i[1:-1] for i in row['SESSIONS_SEQUENCES'][1:-1].split(", ")]

    for i in temp: 
        set_of_hash.add(i)

    a[index] = temp

n_latent = 1

rand_v = np.random.randn(len(set_of_hash), n_latent)

dict_hashes = {}

list_of_hash= list(set_of_hash)

for i, j in zip(range(len(set_of_hash)), list_of_hash):
    dict_hashes[j] = i

In [12]:
processed_vec = [[rand_v[dict_hashes[j]][0] for j in row] for row in a.values()]
token_start = 10
token_end = -10
tokenized_vec = [[token_start] + i + [token_end] for i in processed_vec]

latent_length = 300

post_processed = np.array([np.concatenate((np.zeros((latent_length)), i))[-latent_length:] for i in tokenized_vec])

In [13]:
train = data_utils.TensorDataset(torch.from_numpy(post_processed).float())
train_loader = data_utils.DataLoader(train, batch_size=34, shuffle=True)

In [14]:
class CreditsLSTM(nn.Module):
    def __init__(self, seq_vec_size, rnn_units, top_classifier_units=100, num_out = 100):
        super(CreditsLSTM, self).__init__()
        self.seq_vec_size = seq_vec_size

        self.lstm = nn.LSTM(input_size=seq_vec_size,
                             hidden_size=rnn_units, batch_first=True, bidirectional=False)
                        
        # self._gru = nn.GRU(input_size=seq_vec_size,
        #                      hidden_size=rnn_units, batch_first=True, bidirectional=False)
        
        
        self._hidden_size = rnn_units

        # self.lstm2 = nn.LSTMCell(self.hidden_layers, self.hidden_layers)
        self._top_classifier = nn.Linear(in_features=rnn_units, out_features=num_out)
        # self._intermediate_activation = nn.ReLU()
        # self._head = nn.Linear(in_features=top_classifier_units, out_features=num_out)
    
    def forward(self, seq_vec):

        
        _, last_hidden = self.lstm(seq_vec)
        # print(last_hidden)                                
        classification_hidden = self._top_classifier(last_hidden[0])
        # activation = self._intermediate_activation(classification_hidden)
        # raw_output = self._head(activation)
        return classification_hidden, last_hidden

In [15]:
lstm = CreditsLSTM(seq_vec_size = 300, rnn_units = 150, num_out = 300)
lstm

CreditsLSTM(
  (lstm): LSTM(300, 150, batch_first=True)
  (_top_classifier): Linear(in_features=150, out_features=300, bias=True)
)

In [16]:
optimizer = torch.optim.Adam(lr=1e-3, params=lstm.parameters())
loss_function = torch.nn.L1Loss()
best_loss = 9999999
for epoch in range(1):  # again, normally you would NOT do 300 epochs, it is toy data
    norm = 0
    mae = 0
    val_data = []
    val_error = 0
    for x_batch in train_loader:
        norm +=1
        lstm.zero_grad()
        sequense, val = train_test_split(x_batch[0], test_size=0.1, random_state=42)
        val_data.append(val)
        tag_scores = lstm(sequense)
        loss = loss_function(tag_scores[0], sequense)
        mae += loss.tolist()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        for val in val_data:
            tag_scores = lstm(val)
            loss = loss_function(tag_scores[0], val)
            val_error += loss.tolist()
    if best_loss>val_error/norm:
        best_loss = val_error/norm
        torch.save(lstm.state_dict(), 'lstm_dict.pth')
        torch.save(lstm, 'lstm.pth')
    
    print('Epoha=', epoch+1,' Train loss=', mae/norm, ' Val loss=', val_error/norm)

/home/jupyter-nikita/.local/lib/python3.9/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([30, 300])) that is different to the input size (torch.Size([1, 300])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/jupyter-nikita/.local/lib/python3.9/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([4, 300])) that is different to the input size (torch.Size([1, 300])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoha= 1  Train loss= 0.09075507697188698  Val loss= 0.09063733166419898


In [17]:
torch.save(lstm.state_dict(), 'lstm_dict2.pth')
torch.save(lstm, 'lstm2.pth')

In [ ]:
# func for feaqture for gnn
with torch.no_grad():
    tag_scores = lstm(inputs)
    return tag_scores[1]

In [31]:
a

{0: ['68a9902529901882ecd5143460468d4c',
  '79d75347a9f7b1f2c3a8ab2adc53b572',
  '68a9902529901882ecd5143460468d4c',
  '3b3c39e359f70ab131cf111de8e637fc',
  '68a9902529901882ecd5143460468d4c',
  '3b3c39e359f70ab131cf111de8e637fc',
  '68a9902529901882ecd5143460468d4c',
  '3b3c39e359f70ab131cf111de8e637fc',
  'cd97eac1dc377c5062078f541e6f9cbe',
  '79d75347a9f7b1f2c3a8ab2adc53b572',
  '68a9902529901882ecd5143460468d4c',
  'cd97eac1dc377c5062078f541e6f9cbe',
  '3b3c39e359f70ab131cf111de8e637fc',
  'e4f78c5c57f11e04046d45f35f5f2166',
  'bf5374c9a1231ae744e485fcb39cf703',
  'e4f78c5c57f11e04046d45f35f5f2166',
  'f870f2307d06ffeca80d622296bd2b2c',
  '68a9902529901882ecd5143460468d4c',
  '3b3c39e359f70ab131cf111de8e637fc',
  '68a9902529901882ecd5143460468d4c',
  'cd97eac1dc377c5062078f541e6f9cbe'],
 1: ['72b88f72bca2636844bb14b9cd9e29b3'],
 2: ['b496d19eb25556bd64e481d11b6f7dd6',
  'd21918bb70cf7f728ccdf162a50817c1',
  '1822269b8c1b62e23a1e9db41d92b429',
  '2c15e8a10c7d06c6ac2afb5f74d3eb0a',
 

In [29]:
series = pd.Series(dict_hashes)
series.to_csv('dict_hashes.csv')
series = pd.Series(list(rand_v))
series.to_csv('rand_v.csv')
series = pd.Series(a)
series.to_csv('a.csv')

In [30]:
series = pd.Series(a)
series.to_csv('a.csv')

In [27]:
series = pd.Series(dict_hashes)
series.to_csv('a.csv')

In [26]:
series

                                         0
005ae0147808c294e7c0e80f1a5929c6         1
2e4ea13e69330e4e64471bba01688596         2
1f5ba804a15f5729738edaf549c10713         3
1388cc27373450c353c6c832173a591c         4
                                     ...  
54700aaac22fc641eabc6bd1f45764b9    444059
4a0ae595e24c786153b5ad038e3e8618    444060
dfb1455a1551ddaef12aba43f5c36ce5    444061
244abc490eee4b7882eb3055488c18dd    444062
d0332c194aa7b9507adf4c7f6d6a6712    444063
Length: 444064, dtype: int64

In [32]:
a_temp = pd.read_csv('a.csv')

,Unnamed: 0,0
0,0,"['68a9902529901882ecd5143460468d4c', '79d75347..."
1,1,['72b88f72bca2636844bb14b9cd9e29b3']
2,2,"['b496d19eb25556bd64e481d11b6f7dd6', 'd21918bb..."
3,3,"['dcb3d50673073d3c023db102f21b2d3e', '09a5d816..."
4,4,"['3de59e26ee7e94a6ccd0a2eead2fab9a', '4a51f13a..."
...,...,...
1489571,1489571,['af718335cce0487a7d3b1ae64eb58c32']
1489572,1489572,['2d94c9bf487950e65f6f999ced39c116']
1489573,1489573,['21ab3e195efa4fca5a52a3d9b9eb0253']
1489574,1489574,"['f4d6ee901e5217b0f54350ae716451c2', 'f945d74b..."


In [ ]:
a_temp.

In [21]:
dict_hashes

{'': 0,
 '005ae0147808c294e7c0e80f1a5929c6': 1,
 '2e4ea13e69330e4e64471bba01688596': 2,
 '1f5ba804a15f5729738edaf549c10713': 3,
 '1388cc27373450c353c6c832173a591c': 4,
 '21238ccd6d72f9a42be37700c55ebe9a': 5,
 '585602a068c04688bddb362063eeb51d': 6,
 'dc8560984abd95ef175aec3153e4f094': 7,
 'd1fcf404c723c882673d79717eac5672': 8,
 '70ce31bc0642b76a643b560670526fc2': 9,
 'c6ed2c671696579a937f5eb164f12aba': 10,
 '0dc39f23e31ae3b8886e6a8aca0cddfa': 11,
 '237b286e0da25de618d87bd9bd0b77c4': 12,
 '4eb9ce9bb4b89bf428d2fa792666f546': 13,
 'efb61d7221dec8b92592c960178ca36e': 14,
 '32b675d6cec7fa67a6ae5f0f5356cf0e': 15,
 'd5783020418b78554084c796de8dedba': 16,
 '18057936496b7961232499a823316b69': 17,
 'dc4913d09e0476474860f27a6e11f888': 18,
 'c62089521ca0a3ff41a4d696efb77bf4': 19,
 'bb4164224d3af06b44f28b3c00c6294f': 20,
 'fb8c7de8e5eda1f9df821d5555977c3a': 21,
 'f4d55c6b9cd27e0e619c61a2c4f1115d': 22,
 '70408fc95052ea20436ecbe90b8866f2': 23,
 'c5b7a74075673f571404a84a32ccc5ff': 24,
 '2305f174c817444